<a href="https://colab.research.google.com/github/rahulombale/swing_strategy_rahul/blob/main/V20_Strategy_Daily_Opportunity_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.graph_objects as go
from google.colab import drive
from datetime import datetime
import os

# --- 1. SETUP AND CONFIGURATION ---

# Mount Google Drive
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"Could not mount drive: {e}")

# --- INPUT VARIABLES ---
# This would be updated daily in a real-world scenario
# For now, we use a fixed date to simulate a daily run.
DATA_FOLDER_DATE = '2025-06-17'
DATA_BASE_PATH = '/content/drive/My Drive/stock_data/'
BASE_DATA_PATH = f'{DATA_BASE_PATH}{DATA_FOLDER_DATE}/'

# Output file for the daily signals
OUTPUT_FILE = f'v20_opportunities_{DATA_FOLDER_DATE}.csv'

# Stock list configuration
STOCK_LIST_FILES = [
    'v40_token.csv',
    'v40next_token.csv',
    'v200_token.csv'
]
SOURCE_FILE_MAPPING = {
    'v40_token.csv': 'V40',
    'v40next_token.csv': 'V40Next',
    'v200_token.csv': 'V200'
}
TICKER_COLUMN_NAME = 'ticker'

# Strategy Parameters
SMA_PERIOD = 200
MISSED_TRIGGER_THRESHOLD = 1.10 # 10% above the buy level


# --- 2. CORE FUNCTIONS ---

def load_data(file_path, stock_name_for_print):
    """Loads and preprocesses stock data, including calculating the SMA."""
    if not os.path.exists(file_path):
        print(f"      - WARNING: Data file not found for {stock_name_for_print} at {file_path}. Skipping.")
        return None
    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        df.sort_index(inplace=True)
        df[f'SMA_{SMA_PERIOD}'] = df['close'].rolling(window=SMA_PERIOD).mean()
        return df
    except Exception as e:
        print(f"      - ERROR: Could not process file for {stock_name_for_print}. Reason: {e}")
        return None

def find_v20_setups(df, pct_change_threshold=0.20):
    """Identifies all qualifying v20 setups in the historical data."""
    setups = []
    i = 0
    while i < len(df):
        if i < 1: # Cannot check previous candle if it's the first one
            i+=1
            continue

        # A setup can only start after a red candle or at the beginning
        is_previous_red = df['close'].iloc[i-1] <= df['open'].iloc[i-1]
        is_current_green = df['close'].iloc[i] > df['open'].iloc[i]

        if is_previous_red and is_current_green:
            start_index = i
            j = i
            while j + 1 < len(df) and df['close'].iloc[j+1] > df['open'].iloc[j+1]:
                j += 1
            end_index = j

            segment = df.iloc[start_index:end_index + 1]
            lowest_low = segment['low'].min()
            highest_high = segment['high'].max()

            if (highest_high - lowest_low) / lowest_low > pct_change_threshold:
                setups.append({
                    'start_date': segment.index[0],
                    'end_date': segment.index[-1],
                    'lowest_low': lowest_low,
                    'highest_high': highest_high,
                    'scan_from_index': end_index + 1
                })
            i = end_index + 1
        else:
            i += 1
    return setups

def scan_for_opportunities(df, stock_name, source_file_name):
    """
    Scans a single stock for current V20 trading opportunities.
    """
    opportunities = []

    # We only care about the most recent valid setup
    setups = find_v20_setups(df)
    if not setups:
        return []

    latest_setup = setups[-1]
    buy_level = latest_setup['lowest_low']
    sell_target = latest_setup['highest_high']

    # Analyze data from the point the setup was confirmed
    scan_df = df.iloc[latest_setup['scan_from_index']:]
    if scan_df.empty:
        return []

    # Check the entire history after the setup for a trigger
    triggered_candles = scan_df[scan_df['low'] <= buy_level]

    current_candle = df.iloc[-1]
    current_close = current_candle['close']
    current_low = current_candle['low']

    metadata = {
        'Stock': stock_name,
        'Source File': SOURCE_FILE_MAPPING.get(source_file_name),
        'Setup Start': latest_setup['start_date'].date(),
        'Setup End': latest_setup['end_date'].date(),
        'Buy Level': round(buy_level, 2),
        'Sell Target': round(sell_target, 2),
        'Current Close': round(current_close, 2),
        'Distance From Buy Level %': round(((current_close - buy_level) / buy_level) * 100, 2),
        'Potential Upside %': round(((sell_target - current_close) / current_close) * 100, 2),
        'SMA Filter Status (for V200)': 'N/A'
    }

    # Condition 1: Immediate Trigger Today
    if current_low <= buy_level:
        metadata['Signal Type'] = 'IMMEDIATE_TRIGGER'
        opportunities.append(metadata)

    # Condition 2: Missed Trigger but still nearby
    elif not triggered_candles.empty:
        # Check if the current price is within the 10% threshold
        if buy_level < current_close <= buy_level * MISSED_TRIGGER_THRESHOLD:
            metadata['Signal Type'] = 'MISSED_TRIGGER_NEARBY'
            opportunities.append(metadata)

    # Apply V200 SMA Filter to any found opportunities
    if opportunities and SOURCE_FILE_MAPPING.get(source_file_name) == 'V200':
        sma_value = current_candle[f'SMA_{SMA_PERIOD}']
        if pd.isna(sma_value) or buy_level >= sma_value:
            opportunities[0]['SMA Filter Status (for V200)'] = f'Not Met (Buy Level {buy_level:.2f} >= SMA {sma_value:.2f})'
        else:
            opportunities[0]['SMA Filter Status (for V200)'] = 'Met'

    return opportunities


# --- 3. MAIN EXECUTION BLOCK ---

if __name__ == '__main__':
    all_opportunities = []

    print("--- Starting V20 Daily Opportunity Scanner ---")
    print(f"Scanning data from folder: {DATA_FOLDER_DATE}")

    # Consolidate all tickers from all files into one list
    all_tickers_to_scan = {} # Using dict to store ticker and its source file
    for list_file_name in STOCK_LIST_FILES:
        stock_list_path = os.path.join(DATA_BASE_PATH, list_file_name)
        try:
            stocks_df = pd.read_csv(stock_list_path)
            for ticker in stocks_df[TICKER_COLUMN_NAME].dropna().unique():
                if ticker not in all_tickers_to_scan:
                    all_tickers_to_scan[ticker] = list_file_name
        except FileNotFoundError:
            print(f"  - WARNING: Stock list file not found at {stock_list_path}. Skipping.")

    print(f"Found a total of {len(all_tickers_to_scan)} unique stocks to scan.")

    # Loop through the consolidated list of tickers
    for i, (ticker, source_file) in enumerate(all_tickers_to_scan.items()):
        print(f"  [{i+1}/{len(all_tickers_to_scan)}] Analyzing: {ticker}...", end='\r')

        data_file_path = f"{BASE_DATA_PATH}{ticker}.csv"
        df = load_data(data_file_path, ticker)

        if df is not None and not df.empty:
            opportunities = scan_for_opportunities(df, ticker, source_file)
            if opportunities:
                all_opportunities.extend(opportunities)

    print("\n--- Scan Complete ---")

    # --- 4. FINAL REPORTING ---
    if all_opportunities:
        report_df = pd.DataFrame(all_opportunities)
        # Reorder columns for better readability
        cols_order = ['Stock', 'Source File', 'Signal Type', 'Current Close',
                      'Buy Level', 'Distance From Buy Level %',
                      'Sell Target', 'Potential Upside %', 'SMA Filter Status (for V200)',
                      'Setup Start', 'Setup End']
        report_df = report_df[cols_order]

        # Sort by Signal Type and Distance for priority
        report_df = report_df.sort_values(by=['Signal Type', 'Distance From Buy Level %'], ascending=[False, True])

        output_path = os.path.join(DATA_BASE_PATH, OUTPUT_FILE)
        report_df.to_csv(output_path, index=False)

        print(f"\nSUCCESS: Found {len(report_df)} opportunities.")
        print("Detailed report saved to:")
        print(output_path)
        print("\n" + "="*28 + " OPPORTUNITY SUMMARY " + "="*28)
        print(report_df.to_string())

    else:
        print("\nNo V20 opportunities found for the given date.")

Mounted at /content/drive
--- Starting V20 Daily Opportunity Scanner ---
Scanning data from folder: 2025-06-17
Found a total of 281 unique stocks to scan.

--- Scan Complete ---

SUCCESS: Found 66 opportunities.
Detailed report saved to:
/content/drive/My Drive/stock_data/v20_opportunities_2025-06-17.csv

============================ OPPORTUNITY SUMMARY ============================
            Stock Source File            Signal Type  Current Close  Buy Level  Distance From Buy Level %  Sell Target  Potential Upside %                  SMA Filter Status (for V200) Setup Start   Setup End
16        LICI-EQ        V200  MISSED_TRIGGER_NEARBY         947.45     937.00                       1.12      1145.00               20.85      Not Met (Buy Level 937.00 >= SMA 883.93)  2024-02-02  2024-02-08
60    MAHABANK-EQ        V200  MISSED_TRIGGER_NEARBY          54.44      53.50                       1.76        68.85               26.47        Not Met (Buy Level 53.50 >= SMA 52.59)  2024-01-31 